<div class="alert alert-block alert-info">

## <center> <b> Stock Sentiment </center>
## <center> Predicting market behavior from tweets </center> <br>
##  <center> <b> TRANSFORMER ENCODERS </center> <br>
## <center> Spring Semester 2024-2025 <center>

<center> Group 35: <center>
<center>Joana Esteves, 20240746 <br><center>
<center>José Cavaco, 20240513 <br><center>
<center> Leonardo Di Caterina 20240485<br><center>
<center>Matilde Miguel, 20240549 <br><center>
<center>Rita Serra, 20240515 <br><center>

<div>

# Imports

In [1]:
# General
import numpy as np
import pandas as pd

# Model
import torch.nn.functional as F
import torch
import torch.nn as nn
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, EarlyStoppingCallback, Trainer
from datasets import Dataset, DatasetDict

# Imbalance
from sklearn.utils.class_weight import compute_class_weight

# Evaluation
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.metrics import classification_report
from sklearn.model_selection import StratifiedKFold

import warnings
warnings.filterwarnings("ignore")

seed = 42

/Users/joanaesteves/Desktop/MDSAA-DS/S2/T4/TM/.TM_Project/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import sys
import os
sys.path.append(os.path.abspath('..'))

# Preprocess
from src.preprocessing import PreprocessingPretrained

# Model
from src.tranformer_encoder import TransformerEncoder

In [3]:
# Load data
train_df = pd.read_csv('../Data/train.csv')

# Initialize models

In [4]:
bertweet = "vinai/bertweet-base"
finbert = "yiyanghkust/finbert-tone"

In [5]:
# Initialize models 

Finbert = TransformerEncoder(num_classes=3, model_name=finbert, base_model="BERT")
Bertweet = TransformerEncoder(num_classes=3, model_name=bertweet, base_model="ROBERTA")

# Test pipeline

In [6]:
X_train, X_val, y_train, y_val = train_test_split(
            train_df['text'], train_df['label'], 
            test_size=0.2, stratify=train_df['label'], random_state=seed
        )

In [7]:
# Light preprocessing
preprocessor = PreprocessingPretrained(translate=True)

X_train_prep = preprocessor.preprocess(X_train)
X_val_prep = preprocessor.preprocess(X_val)

In [8]:
Report_Finbert = Finbert.train_predict(X_train_prep, y_train, X_val_prep, y_val)

Map: 100%|██████████| 1909/1909 [00:00<00:00, 29335.42 examples/s]


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,No log,1.027896,0.792038,0.757920,0.700730,0.718998
2,1.043900,0.685775,0.756941,0.691084,0.746419,0.708894
3,0.885900,0.669761,0.804610,0.743891,0.751250,0.747187
4,0.785800,0.628902,0.706129,0.660319,0.761056,0.675595
5,0.715600,0.586313,0.781561,0.712696,0.763430,0.733120


In [9]:
print(Report_Finbert)

              precision    recall  f1-score   support

     bearish       0.58      0.74      0.65       288
     bullish       0.66      0.75      0.70       385
     neutral       0.90      0.80      0.85      1236

    accuracy                           0.78      1909
   macro avg       0.71      0.76      0.73      1909
weighted avg       0.80      0.78      0.79      1909



In [8]:
Report_Bertweet = Bertweet.train_predict(X_train_prep, y_train, X_val_prep, y_val)

Map: 100%|██████████| 1909/1909 [00:00<00:00, 5539.27 examples/s]
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at vinai/bertweet-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,No log,1.059872,0.724463,0.437578,0.520217,0.475235
2,1.819300,1.403098,0.179675,0.385113,0.348166,0.118038
3,1.496900,1.054533,0.563122,0.736489,0.506860,0.406404
4,1.359000,0.913722,0.691461,0.608907,0.560390,0.489438
5,1.173600,0.820468,0.735987,0.626729,0.635606,0.630830


In [9]:
print(Report_Bertweet)

              precision    recall  f1-score   support

     bearish       0.44      0.44      0.44       288
     bullish       0.58      0.62      0.60       385
     neutral       0.86      0.84      0.85      1236

    accuracy                           0.74      1909
   macro avg       0.63      0.64      0.63      1909
weighted avg       0.74      0.74      0.74      1909



# Evaluation

In [ ]:
# Light preprocessing
preprocessor = PreprocessingPretrained(translate=True)

train_df_prep = preprocessor.preprocess(train_df)

X = train_df_prep["text"]
y = train_df_prep["label"]

In [ ]:
def cross_validate_transformer(encoder, X, y, k=5):
    skf = StratifiedKFold(n_splits=k, shuffle=True, random_state=seed)

    macro_precision = []
    macro_recall = []
    macro_f1 = []
    macro_accuracy = []

    for fold, (train_idx, val_idx) in enumerate(skf.split(X, y), 1):

        print(f"Training fold {fold}/{k}...") 
        
        X_train, X_val = X[train_idx], X[val_idx]
        y_train, y_val = y[train_idx], y[val_idx]

        report = encoder.train_predict(X_train, y_train, X_val, y_val)

        macro_precision.append(report['macro avg']['precision'])
        macro_recall.append(report['macro avg']['recall'])
        macro_f1.append(report['macro avg']['f1-score'])
        macro_accuracy.append(report['accuracy'])

    print("\------ Cross-validation results -------")
    print(f"Macro Precision: {np.mean(macro_precision):.4f} ± {np.std(macro_precision):.4f}")
    print(f"Macro Recall:    {np.mean(macro_recall):.4f} ± {np.std(macro_recall):.4f}")
    print(f"Macro F1:        {np.mean(macro_f1):.4f} ± {np.std(macro_f1):.4f}")
    print(f"Accuracy:        {np.mean(macro_accuracy):.4f} ± {np.std(macro_accuracy):.4f}")

In [ ]:
cross_validate_transformer(Finbert)

In [ ]:
cross_validate_transformer(Bertweet)